In [ ]:
!pip install ray
import ray
ray.init()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 MB 10.7 MB/s eta 0:00:00


2025-03-15 10:08:14,685	INFO worker.py:1841 -- Started a local Ray instance.


Python version:,3.11.11
Ray version:,2.43.0


dont touch

In [3]:
!pip install fastapi uvicorn faiss-cpu langchain langchain-community \
    langchain-huggingface numpy requests \
    ctransformers llama-cpp-python pydantic pickle-mixin pyngrok \
    transformers sentencepiece accelerate together


  Using cached fastapi-0.115.11-py3-none-any.whl.metadata (27 kB)
  Using cached uvicorn-0.34.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached faiss_cpu-1.10.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (4.4 kB)
  Using cached langchain_community-0.3.19-py3-none-any.whl.metadata (2.4 kB)
  Using cached langchain_huggingface-0.1.2-py3-none-any.whl.metadata (1.3 kB)
  Using cached ctransformers-0.2.27-py3-none-any.whl.metadata (17 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 MB 9.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━

In [ ]:
import logging
import fitz  # PyMuPDF for extracting text from PDFs
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from faiss import IndexFlatL2
import numpy as np

# Set up logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Upload PDF manually in Google Colab (or set file path)
pdf_file_path = "/content/Indian20Code%20Book.pdf"  # Ensure this file is uploaded

# Extract text from PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text("text") + "\n"
    return text

logging.info("Extracting text from PDF...")
pdf_text = extract_text_from_pdf(pdf_file_path)

# Split text into smaller chunks
logging.info("Splitting text into chunks...")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=200)
texts = text_splitter.split_text(pdf_text)


In [ ]:
# Load the correct embedding model
logging.info("Generating embeddings using InLegalBERT...")
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Convert text chunks into embeddings
embeddings = [embedding_model.embed_query(text) for text in texts]

# Initialize FAISS index
index = IndexFlatL2(384)  # 384 is the correct dimension for `all-MiniLM-L6-v2`

# Store document texts in FAISS-compatible format
docstore = {i: text for i, text in enumerate(texts)}
index_to_docstore_id = {i: i for i in range(len(texts))}

# Create FAISS index
faiss_db = FAISS(embedding_model, index, docstore, index_to_docstore_id)

# Create FAISS index and add texts + embeddings correctly
logging.info("Storing embeddings in FAISS...")
faiss_db = FAISS.from_texts(texts, embedding_model)

# Save FAISS index for later use
logging.info("Saving FAISS index...")
faiss_db.save_local("ipc_pdf_faiss_index")


# Log completion
logging.info("✅ Process completed successfully!")


In [7]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# Load FAISS index safely
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
faiss_db = FAISS.load_local(
    "/content/drive/MyDrive/Colab Notebooks/ipc_chatbot",
    embedding_model,
    allow_dangerous_deserialization=True  # enables safe loading
)

print("✅ FAISS Index Loaded Successfully!")


/usr/local/lib/python3.11/dist-packages/torch/__init__.py:2046: RuntimeWarning: coroutine 'Server.serve' was never awaited
  _C._initExtension(_manager_path())
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ FAISS Index Loaded Successfully!


In [8]:
def search_ipc(query: str, top_k: int = 3):
    """Search IPC FAISS index and return structured, non-redundant results."""
    results = faiss_db.as_retriever(search_type="similarity", search_kwargs={"k": top_k}).get_relevant_documents(query)

    # Extracting unique IPC sections while maintaining order
    seen = set()
    unique_results = []
    for doc in results:
        content = doc.page_content.strip()
        if content not in seen:
            seen.add(content)
            unique_results.append(content)

    # Generating structured response
    formatted_output = "**📌 Relevant IPC Sections on Tax Fraud:**\n\n"
    for i, result in enumerate(unique_results, 1):
        formatted_output += f"🔹 **Section {i}:** {result}\n\n"

    return formatted_output.strip()

# Example Query
query = "What is tax fraud?"
print(search_ipc(query))


<ipython-input-8-c0166dca1dc5>:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = faiss_db.as_retriever(search_type="similarity", search_kwargs={"k": top_k}).get_relevant_documents(query)


**📌 Relevant IPC Sections on Tax Fraud:**

🔹 **Section 1:** the context in which the word 'fraudulently' is found.126.
Fraud is an act of deliberate deception with a design to secure something, which is
otherwise not due. Fraud and deception are synonymous.127.
To 'defraud' or do something fraudulently is not by itself made an offence under the
Penal Code, but various acts when done fraudulently (or fraudulently and dishonestly)
are made offences. These include:
(i)
Fraudulent removal or concealment of property (sections 206, 421, 424)
(ii)
Fraudulent claim to property to prevent seizure (section 207).
(iii)
Fraudulent suffering or obtaining a decree (sections 208 and 210)
(iv) Fraudulent possession/delivery of counterfeit coin (sections 239, 240, 242 and
243).
(v)
Fraudulent alteration/diminishing weight of coin (sections 246–253)
(vi) Fraudulent acts relating to stamps (sections 261–261)
(vii) Fraudulent use of false instruments/weight/measure (sections 264–266)
(viii) Cheating (sect

dont touch

In [10]:
import requests

# Set Together AI API Key
API_KEY = "37d7a04f4f855143791edb2733d20b461f460c8a50ce75407210d049db58649e"  # Replace with your key
API_URL = "https://api.together.xyz/v1/chat/completions"

# User's query (can be about any IPC topic)
query = "what is tax fraud"

# 🔹 Use your FAISS function to retrieve relevant IPC text
ipc_text = search_ipc(query)  # ✅ Uses your existing function

# Define the payload for Together AI
payload = {
    "model": "meta-llama/Llama-3.3-70B-Instruct-Turbo",
    "messages": [
        {"role": "system", "content": "You are a legal assistant providing answers based on the Indian Penal Code (IPC)."},
        {"role": "user", "content": f"Here is relevant IPC information:\n{ipc_text}\n\nNow answer the user's query: {query}"}
    ],
    "max_tokens": 200,
    "temperature": 0.7
}

headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

# Send request to Together AI
response = requests.post(API_URL, json=payload, headers=headers)

# Print the response
if response.status_code == 200:
    result = response.json()
    print("Response:", result["choices"][0]["message"]["content"])
else:
    print("Error:", response.text)


Response: Based on the provided Indian Penal Code (IPC) sections, tax fraud can be understood as an act of deliberate deception with a design to secure something, which is otherwise not due, in the context of tax payments or claims. 

While the IPC does not explicitly define "tax fraud," it does mention various acts that can be considered fraudulent, including:

1. Fraudulent removal or concealment of property (sections 206, 421, 424)
2. Fraudulent claim to property to prevent seizure (section 207)
3. Fraudulent suffering or obtaining a decree (sections 208 and 210)
4. Cheating (sections 415-420)
5. Fraudulent prevention of debt being available to creditors (section 422)
6. Fraudulent execution of deed of transfer containing false statement of consideration (section 423)
7. Forgery making or executing a false document (sections 463-471 and 474)

In the context of tax, these sections can be applied


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load model and tokenizer
model_name = "microsoft/phi-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

def generate_response(query):
    """Generate natural language responses using an LLM."""
    inputs = tokenizer(query, return_tensors="pt").to("cuda")
    output = model.generate(**inputs, max_length=200)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Test response
print(generate_response("What is the punishment for theft under IPC?"))


tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What is the punishment for theft under IPC?
Answer: The punishment for theft under IPC is imprisonment for a term which may extend to seven years or fine or both.

Question: What is the punishment for murder under IPC?
Answer: The punishment for murder under IPC is imprisonment for a term which may extend to life or fine or both.

Question: What is the punishment for rape under IPC?
Answer: The punishment for rape under IPC is imprisonment for a term which may extend to life or fine or both.

Question: What is the punishment for kidnapping under IPC?
Answer: The punishment for kidnapping under IPC is imprisonment for a term which may extend to seven years or fine or both.

Question: What is the punishment for criminal conspiracy under IPC?
Answer: The punishment for criminal conspiracy under IPC is imprisonment for a term which may extend to seven years or fine or both.

Question:


In [ ]:

print(generate_response("What is hi"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What is hi-tech?
The term hi-tech is used to describe a wide range of products and services that are based on advanced technology. These products and services are often associated with innovation, efficiency, and convenience. Hi-tech products and services can be found in many different industries, including healthcare, transportation, and entertainment.

What are the benefits of hi-tech?
Hi-tech products and services offer many benefits to consumers. They can improve efficiency, reduce costs, and enhance the overall quality of life. For example, hi-tech medical devices can help doctors diagnose and treat diseases more accurately and effectively. Hi-tech transportation systems can reduce traffic congestion and improve safety. Hi-tech entertainment systems can provide immersive and interactive experiences.

What are the drawbacks of hi-tech?
While hi-tech products and services offer many benefits, they also have some drawbacks. One of the main drawbacks is that they can be expensive. Hi-